## We are using pretrained model ssd_mobilenet_v3

In [ ]:
# packages we need 
import cv2
import numpy as np

In [2]:
print('we are using cv', cv2.__version__)

we are using cv 4.5.1


### List of objects that can be detected using this model
#### names 

In [3]:
classNames= []
classFilePath = 'ot/coco.names'
with open(classFilePath,'rt') as f:
    classNames = f.read().rstrip('\n').split('\n')

In [4]:
print('Number of objects that can be Detected: ', len(classNames))
print(classNames)

Number of objects that can be Detected:  91
['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'street sign', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'hat', 'backpack', 'umbrella', 'shoe', 'eye glasses', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'plate', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'mirror', 'dining table', 'window', 'desk', 'toilet', 'door', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'blender', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothb

## Creating dnn Detection Model 
locating the trained model class path 

In [5]:
configPath = 'ot/ssd_mobilenet_v3_large_coco_2020_01_14.pbtxt'
weightsPath = 'ot/frozen_inference_graph.pb'

net = cv2.dnn_DetectionModel(weightsPath,configPath)
net.setInputSize(320,320)
net.setInputScale(1.0/ 127.5)
net.setInputMean((127.5, 127.5, 127.5))
net.setInputSwapRB(True)

<dnn_Model 00000254D88E3F90>

## Lets'  test on an image
importing image from assets directory

In [6]:
img = cv2.imread('assets/ocean.jpg')
print(img.shape)

(3041, 5406, 3)


### This image too big for screen, let's Resize it 

In [7]:
# resize a bit 
imgHeight, imgWidth, _ = img.shape
scale= 30
imgH = int (imgHeight *scale /100)
imgW = int (imgWidth * scale /100)
img = cv2.resize(img, (imgW, imgH))

## Lets' declear some variables

In [8]:
rectColor =(244,0,0)
font = cv2.FONT_HERSHEY_COMPLEX
fontSize = 1
fontColor = (200, 200, 0)

<font size=4> Threshold to detect object

In [9]:
thres = 0.45 # Threshold to detect object
nms_thres = .5 # 0-1 higer value means lower suppress 

In [10]:
classIds, confs, bbox = net.detect(img,confThreshold=thres)
type(confs),type(bbox)


(numpy.ndarray, numpy.ndarray)

# Formating for nms, its remove overlap
<fonr size=3> make sure bbox is list<float> and confs is also List<float> ,we dont want anything associate with numpy


In [11]:
bbox = list(bbox)
confs = list(np.array(confs).reshape(1, -1)[0])
confs = list(map(float,confs ))

indics = cv2.dnn.NMSBoxes(bbox, confs, thres, nms_thres) # remove overlaps
print(indics)

[[0]
 [1]
 [2]]


In [12]:
for ind in indics:
    i = ind[0] # ind returns list we need index only
    box = bbox[i]
    
    cv2.rectangle(img, box, rectColor, 1) #set the rectangle around object
    
    #show accuracy level
    cv2.putText(img, str(round(confs[i]*100, 1)), (box[0]+10, box[1]+10), 
               font, fontSize, fontColor, 1)
    #putting name of object 
    cv2.putText(img, classNames[classIds[i][0]-1], (box[0]+70, box[1]+30), 
               font, fontSize, fontColor,1)
    
# Lets show the result 
cv2.imshow('img', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Let's test it on webcam 

In [15]:
import cv2
cam = cv2.VideoCapture(0) # my webcam id
cam.set(3, 600) # width of camView
cam.set(4, 400) # height of camView
cam.set(10, 100) # brightness 

True

In [ ]:
while True:
    
    success, img = cam.read()
    if success:
        classIds, confs, bbox = net.detect(img,confThreshold=thres)
        
        #Formating data
        bbox = list(bbox)
        confs = list(np.array(confs).reshape(1, -1)[0])
        confs = list(map(float,confs ))

        indics = cv2.dnn.NMSBoxes(bbox, confs, thres, nms_thres) # remove overlaps
        
        for ind in indics:
            i = ind[0] # ind returns list we need index only
            box = bbox[i]
    
            cv2.rectangle(img, box, rectColor, 1) #set the rectangle around object
    
            #show accuracy level
            cv2.putText(img, str(round(confs[i]*100, 1)), (box[0]+10, box[1]+10), 
                       font, fontSize, fontColor, 1)
            #putting name of object 
            cv2.putText(img, classNames[classIds[i][0]-1], (box[0]+70, box[1]+30), 
                       font, fontSize, fontColor,1)
        
    
        cv2.imshow("Object detection", img)
    cv2.waitKey(10)
    
cam.release()
cv2.destroyAllWindows()

In [ ]:
cam.release()
cv2.destroyAllWindows()